# Notebook 6 : Gold 2 - Visualisation Finale (PySpark + Matplotlib)

**Durée** : 15 minutes  
**Lakehouse** : Lakehouse_gold  
**Objectif** : Visualiser résultats et interprétation métier

## Cellule 1 : Import bibliothèques

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pyspark.sql import functions as F

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Bibliothèques visualisation importées")

## Cellule 2 : Chargement tables Gold

In [ ]:
df_daily = spark.table("Lakehouse_gold.gold.daily_consumption_by_site").toPandas()
df_kpis = spark.table("Lakehouse_gold.gold.site_kpis").toPandas()
df_type = spark.table("Lakehouse_gold.gold.consumption_by_site_type").toPandas()
df_curtailment = spark.table("Lakehouse_gold.gold.curtailment_opportunities").toPandas()
df_alerts = spark.table("Lakehouse_gold.gold.active_alerts").toPandas()

print(f"📊 Données chargées :")
print(f"  - Daily : {len(df_daily)} lignes")
print(f"  - KPIs : {len(df_kpis)} lignes")
print(f"  - By Type : {len(df_type)} lignes")
print(f"  - Curtailment : {len(df_curtailment)} lignes")
print(f"  - Alerts : {len(df_alerts)} lignes")

## Cellule 3 : Graphique 1 - Consommation par type

In [ ]:
# Agrégation par type
df_by_type_total = df_type.groupby('site_type')['total_consumption_mwh'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = plt.bar(df_by_type_total.index, df_by_type_total.values, color=colors, alpha=0.8, edgecolor='black')

for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f} MWh',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.title('Consommation Totale par Type de Site (30 jours)', fontsize=16, fontweight='bold')
plt.xlabel('Type de Site', fontsize=12)
plt.ylabel('Consommation (MWh)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Graphique 1 : Consommation par type")

## Cellule 4 : Graphique 2 - Facteur de charge par site

In [ ]:
df_kpis_sorted = df_kpis.sort_values('load_factor_pct', ascending=False).head(10)

plt.figure(figsize=(12, 6))
colors_factor = ['#2ECC71' if x > 70 else '#F39C12' if x > 50 else '#E74C3C' 
                 for x in df_kpis_sorted['load_factor_pct']]
bars = plt.barh(df_kpis_sorted['site_id'], df_kpis_sorted['load_factor_pct'], color=colors_factor, alpha=0.8)

plt.axvline(x=100, color='red', linestyle='--', linewidth=2, label='Capacité max (100%)')
plt.axvline(x=70, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Seuil optimal (70%)')

plt.title('Facteur de Charge par Site (Top 10)', fontsize=16, fontweight='bold')
plt.xlabel('Facteur de Charge (%)', fontsize=12)
plt.ylabel('Site', fontsize=12)
plt.xlim(0, 110)
plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Graphique 2 : Facteur de charge")

## Cellule 5 : Graphique 3 - Évolution consommation vs prix (7 derniers jours)

In [ ]:
df_daily['date'] = pd.to_datetime(df_daily['date'])
last_7d = df_daily[df_daily['date'] >= df_daily['date'].max() - pd.Timedelta(days=7)]

daily_agg = last_7d.groupby('date').agg({
    'total_consumption_mwh': 'sum',
    'avg_price_eur_mwh': 'mean'
}).reset_index()

fig, ax1 = plt.subplots(figsize=(14, 6))

color = '#2E86DE'
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Consommation Totale (MWh)', color=color, fontsize=12, fontweight='bold')
line1 = ax1.plot(daily_agg['date'], daily_agg['total_consumption_mwh'], 
                 color=color, marker='o', linewidth=2, markersize=8, label='Consommation')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
color = '#EE5A6F'
ax2.set_ylabel('Prix Spot Moyen (€/MWh)', color=color, fontsize=12, fontweight='bold')
line2 = ax2.plot(daily_agg['date'], daily_agg['avg_price_eur_mwh'], 
                 color=color, marker='s', linewidth=2, markersize=8, linestyle='--', label='Prix Spot')
ax2.tick_params(axis='y', labelcolor=color)

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left', fontsize=11)

plt.title('Consommation vs Prix Spot (7 derniers jours)', fontsize=16, fontweight='bold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("📊 Graphique 3 : Consommation vs Prix")

## Cellule 6 : Graphique 4 - Distribution anomalies par site

In [ ]:
anomalies_by_site = df_daily.groupby('site_id')['nb_anomalies'].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
colors_anomalies = ['#E74C3C' if x > 10 else '#F39C12' if x > 5 else '#95A5A6' 
                    for x in anomalies_by_site.values]
bars = plt.bar(range(len(anomalies_by_site)), anomalies_by_site.values, 
               color=colors_anomalies, alpha=0.8, edgecolor='black')

plt.xticks(range(len(anomalies_by_site)), anomalies_by_site.index, rotation=45, ha='right')
plt.title('Nombre d\'Anomalies Détectées par Site (30 jours)', fontsize=16, fontweight='bold')
plt.xlabel('Site', fontsize=12)
plt.ylabel('Nombre d\'Anomalies', fontsize=12)
plt.axhline(y=10, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Seuil critique (10)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Graphique 4 : Distribution anomalies")

## Cellule 7 : Tableau récapitulatif KPIs

In [ ]:
# KPIs globaux
total_consumption = df_daily['total_consumption_mwh'].sum()
avg_load_factor = df_kpis['load_factor_pct'].mean()
total_anomalies = df_daily['nb_anomalies'].sum()
avg_prediction_error = df_kpis['prediction_error_pct'].mean()
total_curtailment_gain = df_curtailment['potential_gain_eur'].sum()
nb_critical_alerts = len(df_alerts[df_alerts['priority'] == 'CRITIQUE'])

print("\n" + "="*70)
print("📊 TABLEAU DE BORD RÉCAPITULATIF (30 JOURS)")
print("="*70)
print(f"📈 Consommation totale           : {total_consumption:,.0f} MWh")
print(f"⚡ Facteur de charge moyen       : {avg_load_factor:.1f}%")
print(f"⚠️  Anomalies détectées           : {total_anomalies:.0f}")
print(f"🎯 Erreur prédiction moyenne     : {avg_prediction_error:.1f}%")
print(f"💰 Gains effacement potentiels   : {total_curtailment_gain:,.0f} €")
print(f"🔴 Alertes critiques actives     : {nb_critical_alerts}")
print("="*70)

# Top 3 consommateurs
print("\n🏆 TOP 3 CONSOMMATEURS :")
top3 = df_kpis.nlargest(3, 'total_consumption_month_mwh')[['site_id', 'total_consumption_month_mwh']]
for idx, row in top3.iterrows():
    print(f"  {row['site_id']}: {row['total_consumption_month_mwh']:,.0f} MWh")

# Top 3 opportunités effacement
if len(df_curtailment) > 0:
    print("\n💰 TOP 3 OPPORTUNITÉS EFFACEMENT :")
    top3_curtail = df_curtailment.nlargest(3, 'potential_gain_eur')[['site_id', 'hour', 'potential_gain_eur']]
    for idx, row in top3_curtail.iterrows():
        print(f"  {row['site_id']} à {row['hour']}: {row['potential_gain_eur']:,.0f} €")

print("\n" + "="*70)

## Cellule 8 : Interprétation métier finale

### 🎯 Interprétation Finale

**Production totale 30 jours** : Variable selon données générées
- Industrie : ~68% (sites énergivores 24/7)
- Commercial : ~22% (profils 8h-20h semaine)
- Résidentiel : ~10% (pics matin/soir)

**Performance opérationnelle** :
- Facteur de charge moyen : Variable (optimal > 60%)
- Prédictibilité : Erreur variable (bon < 15%)

**Anomalies détectées** : Variable
- Sites à investiguer : Ceux avec >10 anomalies
- Pertes estimées : ~50 MWh × 150 €/MWh = 7,500 €/mois évitables

**Optimisation Demand Response** :
- Opportunités identifiées : Périodes prix >200 €/MWh
- Gains potentiels : Variable selon prix spot
- ROI : Négociation contrats effacement = revenus additionnels

→ **Valeur créée** : Pertes évitées + Revenus effacement = ROI atelier

## Cellule 9 : Résumé final de l'atelier

### ✅ Atelier terminé - Bronze → Silver → Gold

**Ce que vous avez appris** :

#### 📘 Spark SQL (Notebooks 1, 2, 5 partiel)
- ✅ Import et exploration rapide (Bronze)
- ✅ Nettoyage simple (filtres, GROUP BY, jointures)
- ⚠️ **Limite découverte** : Cross-lakehouse avec %%sql = problématique

#### 📗 PySpark (Notebooks 2-6)
- ✅ Accès cross-lakehouse fiable (Lakehouse_name.schema.table)
- ✅ UDF custom pour logique métier (baseline intelligente, z-score)
- ✅ ML avec MLlib (LinearRegression, Pipeline)
- ✅ Visualisations avec Matplotlib (via toPandas)
- ✅ **Solution universelle** : Fonctionne dans tous les cas

#### 🎯 Règle d'or finale (ajustée après tests)

```
┌─────────────────────────────────────────────────────────────┐
│ Dans Fabric avec architecture Bronze/Silver/Gold :          │
│                                                             │
│ ✅ Utilisez TOUJOURS PySpark (pas %%sql) pour :            │
│   - Accès cross-lakehouse                                  │
│   - Tables persistantes (pas de TEMP VIEW)                 │
│   - UDF, ML, logique complexe                              │
│   - Notation 3 parties fiable                              │
│                                                             │
│ ⚠️  %%sql acceptable UNIQUEMENT pour :                     │
│   - Exploration rapide dans UN SEUL lakehouse             │
│   - Requêtes ad-hoc simples                                │
└─────────────────────────────────────────────────────────────┘
```

**Valeur métier démontrée** :
- 🔍 Détection anomalies : Pertes évitées
- 💰 Optimisation effacement : Revenus additionnels
- 📈 Prédiction J+1 : Utilisable pour bidding marché

🎉 **Félicitations ! Vous maîtrisez PySpark dans Microsoft Fabric avec architecture medallion.**